In [1]:
## Importing the librairies

import os
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import json
from typing import Dict, List, Optional, Union
from matplotlib.pyplot import imread
import liana as li
import decoupler as dc
import omnipath
import novae

from gbmhackathon.utils.visium_functions import (
    normalize_anndata_wrapper,
    convert_obsm_to_adata
)
from gbmhackathon.viz.visium_functions import (
    plot_spatial_expression,
    plot_obsm
)
from gbmhackathon.stats.visium_functions import (
    perform_multi_clustering,
    quantify_cell_population_activity
)
from gbmhackathon import MosaicDataset

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 1200

### Hyperparameters choice 



In [18]:
FUNDATION_NOVAE = "MICS-Lab/novae-human-0"
RADIUS_SPATIAL_N = 200

# Retrieving the data from a pretrained model

This part aims to focus on the branch that will be learned using the Spatial-Omics data

In [2]:
visium_dict = MosaicDataset.load_visium(
    sample_list=["HK_G_022a_vis", "HK_G_024a_vis", "HK_G_030a_vis"], # remove this argument to load all available samples)
    resolution="hires"
)


Resolution of the spatial image to load:  hires
You can change the resolution by setting the resolution parameter using the resolution argument.
Loading Visium data, this can take few minutes...


/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, c

In [5]:
visium_dict

{'HK_G_022a_vis': AnnData object with n_obs × n_vars = 2236 × 16927
     obs: 'in_tissue', 'array_row', 'array_col'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'spatial', 'log1p'
     obsm: 'spatial'
     layers: 'raw', 'CPM', 'log_CPM',
 'HK_G_024a_vis': AnnData object with n_obs × n_vars = 4576 × 16927
     obs: 'in_tissue', 'array_row', 'array_col'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'spatial', 'log1p'
     obsm: 'spatial'
     layers: 'raw', 'CPM', 'log_CPM',
 'HK_G_030a_vis': AnnData object with n_obs × n_vars = 4522 × 16927
     obs: 'in_tissue', 'array_row', 'array_col'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'spatial', 'log1p'
     obsm: 'spatial'
     layers: 'raw', 'CPM', 'log_CPM'}

In [3]:
# Normalization of the data
visium_obj = normalize_anndata_wrapper(visium_dict, target_sum=1e6)

In [6]:
visium_obj

{'HK_G_022a_vis': AnnData object with n_obs × n_vars = 2236 × 16927
     obs: 'in_tissue', 'array_row', 'array_col'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'spatial', 'log1p'
     obsm: 'spatial'
     layers: 'raw', 'CPM', 'log_CPM',
 'HK_G_024a_vis': AnnData object with n_obs × n_vars = 4576 × 16927
     obs: 'in_tissue', 'array_row', 'array_col'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'spatial', 'log1p'
     obsm: 'spatial'
     layers: 'raw', 'CPM', 'log_CPM',
 'HK_G_030a_vis': AnnData object with n_obs × n_vars = 4522 × 16927
     obs: 'in_tissue', 'array_row', 'array_col'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'spatial', 'log1p'
     obsm: 'spatial'
     layers: 'raw', 'CPM', 'log_CPM'}

In [10]:
# Getting the Ann_datas as a list
list_ids = list(visium_obj.keys())
list_ids

['HK_G_022a_vis', 'HK_G_024a_vis', 'HK_G_030a_vis']

In [14]:
list_anndata = list(visium_obj.values())
list_anndata[0]

AnnData object with n_obs × n_vars = 2236 × 16927
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial', 'log1p'
    obsm: 'spatial'
    layers: 'raw', 'CPM', 'log_CPM'

In [16]:
# Getting the spatial neighbors of each spatial
novae.utils.spatial_neighbors(list_anndata, radius=RADIUS_SPATIAL_N)

[INFO] (novae.utils._build) Computing graph on 2,236 cells (coord_type=generic, delaunay=True, radius=[0.0, 200.0], n_neighs=None)
[INFO] (novae.utils._build) Computing graph on 4,576 cells (coord_type=generic, delaunay=True, radius=[0.0, 200.0], n_neighs=None)
[INFO] (novae.utils._build) Computing graph on 4,522 cells (coord_type=generic, delaunay=True, radius=[0.0, 200.0], n_neighs=None)


In [17]:
list_anndata[0]

AnnData object with n_obs × n_vars = 2236 × 16927
    obs: 'in_tissue', 'array_row', 'array_col', 'novae_sid'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial', 'log1p', 'spatial_neighbors'
    obsm: 'spatial'
    layers: 'raw', 'CPM', 'log_CPM'
    obsp: 'spatial_connectivities', 'spatial_distances'

## Importation of the fundation model

In [28]:
model = novae.Novae.from_pretrained(FUNDATION_NOVAE)
print(model)

Novae model
   ├── Known genes: 60697
   ├── Parameters: 32.0M
   └── Model name: MICS-Lab/novae-human-0


In [21]:
model.compute_representations(list_anndata) 

[WARNING] (novae.utils._validate) adata.X has high values. We recommend processing the data before running the model (e.g., with sc.pp.log1p).
[WARNING] (novae.utils._validate) adata.X has high values. We recommend processing the data before running the model (e.g., with sc.pp.log1p).
[WARNING] (novae.utils._validate) adata.X has high values. We recommend processing the data before running the model (e.g., with sc.pp.log1p).
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Computing representations:   0%|          | 0/5 [00:00<?, ?it/s]

Computing representations:   0%|          | 0/9 [00:00<?, ?it/s]

Computing representations:   0%|          | 0/9 [00:00<?, ?it/s]

In [18]:
list_anndata[0]

NameError: name 'list_anndata' is not defined

In [24]:
list_latent = [anndata.obsm["novae_latent"] for anndata in list_anndata]

In [26]:
dict_latent = dict(zip(list_ids, list_latent))

In [27]:
dict_latent

{'HK_G_022a_vis': array([[-0.26502377, -0.06550228,  0.34204012, ..., -0.2073409 ,
         -0.10982005, -0.03834625],
        [-0.36333096,  0.02693802,  0.48565754, ..., -0.23571926,
         -0.21825948, -0.12070286],
        [-0.16917296, -0.24793334,  0.16761912, ..., -0.28157625,
         -0.28407103,  0.05712069],
        ...,
        [ 0.1000764 , -0.0395981 ,  0.16909938, ..., -0.37550184,
         -0.44573182,  0.09734674],
        [ 0.14575958, -0.12620066,  0.20363583, ..., -0.43433014,
         -0.39884344, -0.00474053],
        [-0.30047482, -0.00350424,  0.4448367 , ..., -0.39232376,
         -0.29894692, -0.14154384]], dtype=float32),
 'HK_G_024a_vis': array([[ 0.33391577, -0.14146209,  0.02248746, ..., -0.31324652,
         -0.5771508 ,  0.12117112],
        [-0.41974166, -0.09113917,  0.29026487, ..., -0.16136858,
         -0.03506573, -0.21532953],
        [ 0.20964031, -0.07997781,  0.05296602, ..., -0.37983233,
         -0.6108271 ,  0.04950904],
        ...,
     

In [2]:
from utils_gbm import retrieve_visium_embedding

In [3]:
dict_emb_test = retrieve_visium_embedding(radius_spatial_n = 300)

No sample list provided. Loading all samples.
Resolution of the spatial image to load:  hires
You can change the resolution by setting the resolution parameter using the resolution argument.
Loading Visium data, this can take few minutes...


/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/ec2-user/SageMaker/.conda/envs/gbmhackathon/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, c

Considered list id : 
 ['HK_G_001a_vis', 'HK_G_002a_vis', 'HK_G_003a_vis', 'HK_G_004a_vis', 'HK_G_005a_vis', 'HK_G_006a_vis', 'HK_G_007a_vis', 'HK_G_008a_vis', 'HK_G_009a_vis', 'HK_G_010a_vis', 'HK_G_011a_vis', 'HK_G_012a_vis', 'HK_G_013a_vis', 'HK_G_014a_vis', 'HK_G_015a_vis', 'HK_G_016a_vis', 'HK_G_017b_vis', 'HK_G_018a_vis', 'HK_G_019a_vis', 'HK_G_020a_vis', 'HK_G_021a_vis', 'HK_G_022a_vis', 'HK_G_023a_vis', 'HK_G_024a_vis', 'HK_G_025a_vis', 'HK_G_026a_vis', 'HK_G_027a_vis', 'HK_G_028a_vis', 'HK_G_029b_vis', 'HK_G_030a_vis', 'HK_G_031a_vis', 'HK_G_032a_vis', 'HK_G_033a_vis', 'HK_G_034a_vbu', 'HK_G_035a_vis', 'HK_G_036b_vis', 'HK_G_037a_vis', 'HK_G_039a_vis', 'HK_G_040a_vbu', 'HK_G_041a_vis', 'HK_G_042a_vis', 'HK_G_043a_vis', 'HK_G_044b_vis', 'HK_G_045a_vis', 'HK_G_046a_vis', 'HK_G_047a_vis', 'HK_G_048a_vis', 'HK_G_049a_vis', 'HK_G_050a_vis', 'HK_G_051a_vis', 'HK_G_052a_vis', 'HK_G_053a_vis', 'HK_G_054a_vis', 'HK_G_055a_vis', 'HK_G_056a_vis', 'HK_G_057a_vis', 'HK_G_058a_vis', 'HK_G_0

[INFO] (novae.utils._build) Computing graph on 2,712 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 3,846 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 4,095 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 2,236 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 4,663 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 4,576 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 4,240 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_neighs=6)
[INFO] (novae.utils._build) Computing graph on 4,480 cells (coord_type=grid, delaunay=False, radius=[0.0, 300.0], n_ne

DONE
___ Importation of the pretrained model ___


...

Computing representations:   0%|          | 0/10 [00:00<?, ?it/s]

Computing representations:   0%|          | 0/9 [00:00<?, ?it/s]

Computing representations:   0%|          | 0/9 [00:00<?, ?it/s]

Computing representations:   0%|          | 0/6 [00:00<?, ?it/s]

DONE
Size latent space = 64


In [4]:
dict_emb_test

{'HK_G_001a_vis': array([[-0.31912524, -0.11272641,  0.2837144 , ..., -0.09955969,
          0.03288715, -0.36776334],
        [-0.3330066 , -0.03813721,  0.26050317, ..., -0.15780449,
          0.03129857, -0.29435375],
        [ 0.12384544,  0.11602774,  0.44294336, ..., -0.402109  ,
         -0.19718835, -0.04662852],
        ...,
        [-0.23171404, -0.13475265,  0.3546657 , ..., -0.05644633,
          0.12796688, -0.40540335],
        [ 0.34226498,  0.10448818,  0.3366263 , ..., -0.48089442,
         -0.24782735, -0.01127741],
        [-0.34072948, -0.00999007,  0.27253863, ..., -0.10768241,
          0.00071271, -0.33029935]], dtype=float32),
 'HK_G_002a_vis': array([[-0.26963663, -0.07992981,  0.18574283, ..., -0.10479557,
         -0.04408744, -0.3014545 ],
        [-0.17910966,  0.03044075,  0.2845866 , ..., -0.32706526,
         -0.30947345, -0.17104314],
        [-0.11633328,  0.00402572,  0.19869089, ..., -0.1756422 ,
         -0.13632794, -0.2201111 ],
        ...,
     

In [6]:
import pickle

In [8]:
with open("novae_emb_dict.pkl","wb") as f :
    pickle.dump(dict_emb_test,f)

In [10]:
for value in dict_emb_test.values():
    print(value.shape)

(1462, 64)
(4467, 64)
(2876, 64)
(2237, 64)
(3589, 64)
(4832, 64)
(4773, 64)
(4672, 64)
(4583, 64)
(2771, 64)
(3166, 64)
(3209, 64)
(3500, 64)
(1472, 64)
(2464, 64)
(3233, 64)
(4401, 64)
(3604, 64)
(2712, 64)
(3846, 64)
(4095, 64)
(2236, 64)
(4663, 64)
(4576, 64)
(4240, 64)
(4480, 64)
(4829, 64)
(2900, 64)
(4502, 64)
(4522, 64)
(2391, 64)
(3856, 64)
(4672, 64)
(4175, 64)
(4692, 64)
(3808, 64)
(4513, 64)
(2732, 64)
(4656, 64)
(2174, 64)
(3708, 64)
(4395, 64)
(4866, 64)
(4783, 64)
(4362, 64)
(4474, 64)
(4247, 64)
(2529, 64)
(3295, 64)
(3446, 64)
(4939, 64)
(3750, 64)
(4355, 64)
(4880, 64)
(1254, 64)
(2988, 64)
(4256, 64)
(4730, 64)
(4349, 64)
(3315, 64)
(4520, 64)
(4662, 64)
(3554, 64)
(3923, 64)
(2252, 64)
(4564, 64)
(4860, 64)
(2702, 64)
(3593, 64)
(2961, 64)
(4760, 64)
(3643, 64)
(1692, 64)
(4524, 64)
(1388, 64)
(3152, 64)
(4350, 64)
(3653, 64)
(4819, 64)
(4546, 64)
(4211, 64)
(4177, 64)
(4796, 64)
(4578, 64)
(4170, 64)
(2570, 64)


In [14]:
for i in range (len(dict_emb_test)):
    novae.plot.domains(dict_emb_test[i],cell_size=200)

KeyError: 0

In [12]:
import novae

In [17]:
for i in dict_emb_test.keys():
 print("Key"+" "+i)
novae.plot.domains(one_visium_file,cell_size=200)


Key HK_G_001a_vis
Key HK_G_002a_vis
Key HK_G_003a_vis
Key HK_G_004a_vis
Key HK_G_005a_vis
Key HK_G_006a_vis
Key HK_G_007a_vis
Key HK_G_008a_vis
Key HK_G_009a_vis
Key HK_G_010a_vis
Key HK_G_011a_vis
Key HK_G_012a_vis
Key HK_G_013a_vis
Key HK_G_014a_vis
Key HK_G_015a_vis
Key HK_G_016a_vis
Key HK_G_017b_vis
Key HK_G_018a_vis
Key HK_G_019a_vis
Key HK_G_020a_vis
Key HK_G_021a_vis
Key HK_G_022a_vis
Key HK_G_023a_vis
Key HK_G_024a_vis
Key HK_G_025a_vis
Key HK_G_026a_vis
Key HK_G_027a_vis
Key HK_G_028a_vis
Key HK_G_029b_vis
Key HK_G_030a_vis
Key HK_G_031a_vis
Key HK_G_032a_vis
Key HK_G_033a_vis
Key HK_G_034a_vbu
Key HK_G_035a_vis
Key HK_G_036b_vis
Key HK_G_037a_vis
Key HK_G_039a_vis
Key HK_G_040a_vbu
Key HK_G_041a_vis
Key HK_G_042a_vis
Key HK_G_043a_vis
Key HK_G_044b_vis
Key HK_G_045a_vis
Key HK_G_046a_vis
Key HK_G_047a_vis
Key HK_G_048a_vis
Key HK_G_049a_vis
Key HK_G_050a_vis
Key HK_G_051a_vis
Key HK_G_052a_vis
Key HK_G_053a_vis
Key HK_G_054a_vis
Key HK_G_055a_vis
Key HK_G_056a_vis
Key HK_G_0

In [16]:
dict_emb_test[0]

KeyError: 0